In [ ]:
!pip install datasets

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [ ]:
## source of our data used in this project >> "https://data.mendeley.com/datasets/6k97jty9xg/5" / but the .CSV file is a preprocessed version by us
dataset = load_dataset("csv", data_files="/content/EN-EGP.csv")
dataset = dataset["train"].train_test_split(test_size=0.1, shuffle=True, seed=42)  # Added seed for reproducibility
dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 20179
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 2243
    })
})

In [ ]:
## showcasing a sapmle of the data we used
for pair in dataset['train'].select(range(5)):
    print(f"English sentence: {pair['source']}\nEgyptian Arabic translation: {pair['target']}\n")

print(f"# of translations in the dataset: {len(dataset['train'])} — relatively small for this use case, but sufficient as a base model for now.") ## only the size of the "Train" split..

English sentence: He leaves the place.
Egyptian Arabic translation: بيهرب من المكان

English sentence: You've got exams and need to study, and your dad needs to rest.
Egyptian Arabic translation: ورانا مذاكرة كتير والامتحانات خلاص قربت، ولازم بابا يريح شوية.

English sentence: I don't watch TV that much. 
Egyptian Arabic translation: انا بس مبتفرجش على التلفزيون. 

English sentence: Seriously, though. I can't believe I made it here, the camp for men only. I can't believe it.
Egyptian Arabic translation: لا لا، أنا مش مصدقة أنا إزاي أصلًا دخلت هنا، أنا مش مصدقة.

English sentence: No, no! Wait, Ola! Wait, wait! Why are you being so hasty?
Egyptian Arabic translation: لا هو ايه ما خلاص ايه بس؟ 

# of translations in the dataset: 20179 — relatively small for this use case, but sufficient as a base model for now.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

SRC_LANG = "eng_Latn"
TGT_LANG = "arb_Arab"
tokenizer.src_lang = SRC_LANG

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def translate(text, max_length=128):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)

    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(TGT_LANG),
        max_length=max_length
    )

    return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

english_text = "now model all it can do is just translate to the Modern Standard Arabic version"
arabic_translation = translate(english_text)
print(arabic_translation)

الآن النموذج كل ما يمكنه فعله هو الترجمة إلى النسخة العربية المعاصرة


In [ ]:
def dataset_tokenization(batch):
    ## source language id for encoder
    model_inputs = tokenizer(
        batch["source"],  ## English translation
        max_length=128,
        padding="max_length",
        truncation=True
    )

    ## target language id for decoder
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(
        batch["target"],  ## Egyptian Arabic target (desired to translate to, later)
        max_length=128,
        padding="max_length",
        truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(dataset_tokenization, batched=True, remove_columns=dataset["train"].column_names)
tokenized_dataset

Map:   0%|          | 0/2243 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20179
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2243
    })
})

In [ ]:
!pip install bitsandbytes

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print("Loading quantized model...")
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,  ## this just uses the model_name variable defined earlier
    quantization_config=bnb_config,
    torch_dtype=torch.float16
).to(device)
print("Loaded!!")

Loading quantized model...


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
model = prepare_model_for_kbit_training(model) ## preparing out "nllb" translation model for QLoRA

## the LoRA config - Modified for better coverage and learning
lora_config = LoraConfig(
    r=8,  ## reduced rank for faster training but loss in performance is the cost here
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM ## since obviously our task is "Translation", it's set to Sequence-to-Sequence
)

model = get_peft_model(model, lora_config)
model.config.use_cache = False
model.print_trainable_parameters()

In [ ]:
## now just setting up the "training" arguments + the "data collator" - Optimized for FAST training
training_args = Seq2SeqTrainingArguments(
    output_dir="./nllb-egyptian-arabic-lora-full",
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=0.5,              ## training for half an epoch (10 hours (precalculated from a simpler run))
    weight_decay=0.01,
    warmup_ratio=0.05,
    eval_strategy="steps",             ## changed it to steps-based evaluation
    eval_steps=0.1,                    ## evaluating every 10% of an epoch
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="steps",             ## saving based on steps
    save_steps=0.1,                    ## saving every 10% of an epoch
    save_total_limit=3,
    predict_with_generate=True,
    generation_max_length=128,
    fp16=True,
    push_to_hub=False,
    optim="adamw_bnb_8bit",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': False},
    no_cuda=False,
    dataloader_num_workers=0,
    report_to="tensorboard",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,  # Crucial: Ignore padding tokens in loss calculation
    pad_to_multiple_of=8      # Pad sequences to a multiple of 8 (can improve hardware efficiency)
)
model.train()

In [ ]:
## finally initilaizing the trainingggg...
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

In [ ]:
print("Starting training...")
trainer.train()
print("\nTraining finished successfully!")

print("\nSaving the final LoRA adapter model...")
final_save_path = os.path.join(training_args.output_dir, "final_lora_adapter")
model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)
print(f"Final LoRA adapter saved to: {final_save_path}")

## Training Loss Curve:

![image.png](attachment:image.png)

## Eval Loss Curve:

![image-2.png](attachment:image-2.png)

(from TensorBoard)

In [ ]:
finetuned_path = "./nllb-egyptian-arabic-lora-full/final_lora_adapter"
tokenizer = AutoTokenizer.from_pretrained(finetuned_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_model_name = "facebook/nllb-200-distilled-600M"
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16
).to(device)

## here we are loading the LoRA adapter onto the base model
model = PeftModel.from_pretrained(base_model, finetuned_path).to(device)
model.eval()

In [ ]:
def translate_text(text, model, tokenizer, lang):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    translated = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(lang),
        max_length=128
    )
    return tokenizer.batch_decode(translated, skip_special_tokens=True)[0]

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def gradio_translate(text):
    base_translation = translate_text(text, model, tokenizer, "arb_Arab")
    finetuned_translation = translate_text(text, base_model, tokenizer, "arz_Arab")
    return base_translation, finetuned_translation

    return base_translation, finetuned_translation
demo = gr.Interface(
    fn=gradio_translate,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter text here..."),
    ],
    outputs=[
        gr.Textbox(label="Base Translated Text"),
        gr.Textbox(label="Finetuned Translated Text")
    ],
    live=True
)

demo.launch(debug=True)